In [ ]:
# IMPORTS
import json
import pandas as pd

# Load JSON file (adjust path as needed in Colab)
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name) as f:
    data = json.load(f)

In [ ]:
# Ground Truth Definitions
ground_truths = {
    "hypothex": {
        "importance": ["f5", "f4", "f1", "f2", "f3"],
        "value_ranges": {
            "f5": (8, 15),  # Tool Worship >7
            "f4": (0, 4),   # Box Collecting <4
            "f1": (0, 5),   # LPC <5
            "f2": (0, 5),   # RSS <5
            "f3": (5, 8)    # PTP 5–8
        },
        "main_profile": "Tool-Worship Focused Zarnak",
        "sub_profiles": [
            "Low Activity Zarnak",
            "Balanced Worship + PTP Zarnak"
        ]
    },
    "whatif": {
        "importance": ["f5", "f2", "f1", "f3", "f4"],
        "value_ranges": {
            "f5": (0, 4),    # Gardening < 4
            "f2": (5, 10),   # Swimming 5–10
            "f1": (5, 9),    # Running 5–9
            "f3": (6, 12),   # Art 6–12
            "f4": (7, 9)     # Detective Stories 7–9
        },
        "main_profile": "General Duck Logic",
        "sub_profiles": [
            "Aquatic Nature Duck",
            "Creative Aquatic Duck",
            "Logical Low-Activity Duck"
        ]
    }
}

# 🧮 Label → numeric range mapping
label_ranges = {
    "low": (0, 5),
    "medium-low": (0, 10),
    "medium": (5, 10),
    "medium-high": (5, 15),
    "high": (10, 15)
}

# ⚙️ Metric Functions
def score_importance_match(user_list, truth_list):
    if not user_list:
        return 0.0
    jaccard = len(set(user_list).intersection(truth_list)) / len(set(user_list).union(truth_list))
    order_penalty = sum([abs(i - truth_list.index(f)) for i, f in enumerate(user_list) if f in truth_list])
    max_penalty = len(truth_list) * len(truth_list)
    order_score = 1 - (order_penalty / max_penalty if max_penalty > 0 else 0)
    return round((0.5 * jaccard + 0.5 * order_score), 2)

def range_overlap_score(user_label, gt_range):
    user_range = label_ranges.get(user_label.lower())
    if not user_range or not gt_range:
        return 0.0
    u_start, u_end = user_range
    g_start, g_end = gt_range

    overlap_start = max(u_start, g_start)
    overlap_end = min(u_end, g_end)
    overlap = max(0, overlap_end - overlap_start)

    user_len = u_end - u_start
    return round(overlap / user_len, 2) if user_len > 0 else 0.0

def score_value_direction_match(user_values, task):
    if not user_values:
        return 0.0
    gt_ranges = ground_truths[task]["value_ranges"]
    scores = []
    for f, label in user_values.items():
        gt_range = gt_ranges.get(f)
        score = range_overlap_score(label, gt_range)
        scores.append(score)
    return round(sum(scores) / len(gt_ranges), 2) if scores else 0.0

def profile_match_score(profile, main, subs):
    if profile == main:
        return 1.0
    elif profile in subs:
        return 0.5
    return 0.0

def overall_correctness_score(importance, value, profile):
    return round(0.4 * importance + 0.4 * value + 0.2 * profile, 2)

# Evaluation Loop
results = []
for task in ["hypothex", "whatif"]:
    for pid, response in data["task1"][task].items():
        imp_score = score_importance_match(response["importance_order"], ground_truths[task]["importance"])
        val_score = score_value_direction_match(response["value_ranges"], task)
        prof_key = "matches_profile" if task == "hypothex" else "matching_profile"
        prof_score = profile_match_score(response.get(prof_key), ground_truths[task]["main_profile"], ground_truths[task]["sub_profiles"])
        overall = overall_correctness_score(imp_score, val_score, prof_score)

        results.append({
            "tool_name": task,
            "participant_id": pid,
            "importance_score": imp_score,
            "value_score": val_score,
            "profile_score": prof_score,
            "overall_score": overall
        })

# 📊 Show Results
df_task1 = pd.DataFrame(results)
df_task1.sort_values(by=["tool_name", "participant_id"], inplace=True)
df_task1.reset_index(drop=True, inplace=True)
df_task1.to_csv("evaluation_results_task_1.csv", index=False)
df_task1

In [ ]:
# 📊 STATISTICS PER TOOL AND PER PARTICIPANT

# 1. Summary statistics per tool (mean scores across all participants)
tool_summary = df_task1.groupby("tool_name").agg({
    "importance_score": "mean",
    "value_score": "mean",
    "profile_score": "mean",
    "overall_score": "mean"
}).reset_index()

print("📈 Average Scores per Tool:")
print(tool_summary)

# Group correctness scores per participant for ANOVA
participant_scores = df_task1.groupby("participant_id")["overall_score"].apply(list).reset_index()
print(participant_scores)

# 2. Per-participant overall score for each tool
participant_summary = df_task1.pivot(index="participant_id", columns="tool_name", values="overall_score").reset_index()
participant_summary.columns.name = None  # Clean display

print("\n📊 Per-Participant Overall Scores (per Tool):")
print(participant_summary)


In [ ]:
# === Define scoring functions ===
def compute_correctness_score(labels):
    label_weights = {
        "refutes_hypothesis": 1.0,
        "works_in_combination": 1.0,
        "weak_influence": 1.0,
        "no_effect": 0.75,
        "ambiguous": 0.5
    }
    scores = [label_weights[lbl] for lbl in labels if lbl in label_weights]
    return round(sum(scores) / len(scores), 2) if scores else 0.0


def compute_analysis_score(evidence):
    score = 0
    if evidence.get("tested_multiple_classes"):
        score += 0.3
    if evidence.get("tried_disconfirming_cases"):
        score += 0.2
    if evidence.get("working_combination_found"):
        score += 0.3
    if score > 0:
        score += 0.2  # reasoning bonus
    return min(score, 1.0)

# === Build the result rows ===
task2_results = []
for tool in data["task2"]:
    for pid, entry in data["task2"][tool].items():
        labels = entry.get("hypothesis_validation_labels", [])
        evidence = entry.get("evidence_base", {})
        correctness = compute_correctness_score(labels)
        analysis = compute_analysis_score(evidence)
        final_score = round(0.4 * correctness + 0.6 * analysis, 2)

        task2_results.append({
            "tool": tool,
            "participant_id": pid,
            "correctness_score": correctness,
            "analysis_score": analysis,
            "overall_score": final_score
        })

# === Create output DataFrame ===
df_task2 = pd.DataFrame(task2_results)
df_task2.sort_values(by=["tool", "participant_id"], inplace=True)
df_task2.reset_index(drop=True, inplace=True)

# === Save & show ===
df_task2.to_csv("task2_scores_output.csv", index=False)
df_task2


In [ ]:
# 📊 1. Average scores per tool
tool_summary = df_task2.groupby("tool").agg({
    "correctness_score": "mean",
    "analysis_score": "mean",
    "overall_score": "mean"
}).reset_index()

print("📈 Average Scores per Tool (Task 2):")
print(tool_summary)

# 📊 2. Per-participant final score for each tool
participant_summary = df_task2.pivot(index="participant_id", columns="tool", values="overall_score").reset_index()
participant_summary.columns.name = None  # clean display

print("\n📊 Per-Participant Final Scores (Task 2):")
print(participant_summary)


In [ ]:
# ⚙️ Metric Functions for Correctness and Complexity

# Correctness Scoring based on the discrepancy of labels between truth and participant
def calculate_correctness_score(ground_truth, participant_evaluation):
    score_map = {
        "confirmed": 0,
        "weakly_confirmed": 0.25,
        "partially_disproved": 0.5,
        "weakly_disproved": 0.75,
        "disproved": 1
    }

    # Get the numerical values for ground truth and participant evaluation
    ground_value = score_map.get(ground_truth, 0)
    participant_value = score_map.get(participant_evaluation, 0)

    # Calculate the final score
    return 1 - abs(ground_value - participant_value)


# Complexity Evaluation based on weighted sum
def calculate_complexity_score(hypothesis_property):
    w_boundary = 0.4
    w_range = 0.3
    w_feature = 0.2
    w_classes = 0.1

    # Get the values from the hypothesis_property
    boundary_score = hypothesis_property["number_of_boundary_values"]
    range_score = hypothesis_property["number_of_ranges_involved"]
    feature_score = hypothesis_property["number_of_features"]
    classes_score = hypothesis_property["number_of_classes"]

    # Calculate the weighted complexity score
    complexity_score = (w_boundary * boundary_score +
                        w_range * range_score +
                        w_feature * feature_score +
                        w_classes * classes_score)

    # Define the maximum possible values for each component
    max_boundary_value = 4  # max number of boundary values (example: >5, <4, etc.)
    max_range_value = 3     # max number of ranges (example: low, medium, high)
    max_feature_value = 5   # max number of features considered
    max_class_value = 3     # max number of classes (example: Dog, Duck, Quorvian)

    # Calculate the max possible complexity score (when all components are at their maximum)
    max_possible_complexity = (w_boundary * max_boundary_value +
                               w_range * max_range_value +
                               w_feature * max_feature_value +
                               w_classes * max_class_value) * 0.6

    # Normalize the complexity score to a 0 - 1 range
    normalized_complexity_score = complexity_score / max_possible_complexity

    # Return the complexity level based on the normalized score
    if normalized_complexity_score <= 0.33:
        return "Low Complexity", normalized_complexity_score
    elif normalized_complexity_score <= 0.66:
        return "Moderate Complexity", normalized_complexity_score
    else:
        return "High Complexity", normalized_complexity_score


# Evaluation Loop for Complexity and Correctness
# Evaluation Loop for Complexity and Correctness
results = []
for tool in ["hypothex", "whatif"]:
    for pid, response in data["task3"][tool].items():
        # Ensure the tool is explicitly added to the result
        complexity_level, complexity_score = calculate_complexity_score(response["hypothesis_property"])

        # Calculate Correctness Score
        ground_truth = response["hypothesis_validation_based_on_ground_true"]["value"]
        participant_evaluation = response["hypothesis_evaluation_by_participant"]["value"]
        correctness_score = calculate_correctness_score(ground_truth, participant_evaluation)

        # Calculate Final Combined Score
        final_score = (0.4 * complexity_score) + (0.6 * correctness_score)

        # Add evaluation results to the list with the tool field
        results.append({
            "tool": tool,  # Explicitly add tool information
            "participant_id": pid,
            "complexity_score": complexity_score,
            "correctness_score": correctness_score,
            "overall_score": final_score,
        })

# 📊 Show Results
df_task3 = pd.DataFrame(results)
df_task3.sort_values(by=["tool", "participant_id"], inplace=True)
df_task3.reset_index(drop=True, inplace=True)
df_task3.to_csv("evaluation_results_task_3.csv", index=False)

df_task3



In [ ]:
# Ensure that the columns have numeric values (not strings)
df_task3["correctness_score"] = pd.to_numeric(df_task3["correctness_score"], errors="coerce")
df_task3["complexity_score"] = pd.to_numeric(df_task3["complexity_score"], errors="coerce")
df_task3["overall_score"] = pd.to_numeric(df_task3["overall_score"], errors="coerce")

# Now perform the groupby operation for average scores per tool
tool_summary = df_task3.groupby("tool").agg({
    "correctness_score": "mean",
    "complexity_score": "mean",
    "overall_score": "mean"
}).reset_index()

print("📈 Average Scores per Tool (Task 3):")
print(tool_summary)


In [ ]:
# Evaluation Loop for Complexity and Correctness
results = []
for tool in ["hypothex", "whatif"]:
    for pid, response in data["task4"][tool].items():
        # Ensure the tool is explicitly added to the result
        complexity_level, complexity_score = calculate_complexity_score(response["hypothesis_property"])

        # Calculate Correctness Score
        ground_truth = response["hypothesis_validation_based_on_ground_true"]["value"]
        participant_evaluation = response["hypothesis_evaluation_by_participant"]["value"]
        correctness_score = calculate_correctness_score(ground_truth, participant_evaluation)

        # Calculate Final Combined Score
        final_score = (0.4 * complexity_score) + (0.6 * correctness_score)

        # Add evaluation results to the list with the tool field
        results.append({
            "tool": tool,  # Explicitly add tool information
            "participant_id": pid,
            "complexity_score": complexity_score,
            "correctness_score": correctness_score,
            "overall_score": final_score,
        })

# 📊 Show Results
df_task4 = pd.DataFrame(results)
df_task4.sort_values(by=["tool", "participant_id"], inplace=True)
df_task4.reset_index(drop=True, inplace=True)
df_task4.to_csv("evaluation_results_task_4.csv", index=False)

df_task4



In [ ]:
# Ensure that the columns have numeric values (not strings)
df_task4["correctness_score"] = pd.to_numeric(df_task4["correctness_score"], errors="coerce")
df_task4["complexity_score"] = pd.to_numeric(df_task4["complexity_score"], errors="coerce")
df_task4["overall_score"] = pd.to_numeric(df_task4["overall_score"], errors="coerce")

# Now perform the groupby operation for average scores per tool
tool_summary = df_task4.groupby("tool").agg({
    "correctness_score": "mean",
    "complexity_score": "mean",
    "overall_score": "mean"
}).reset_index()

print("📈 Average Scores per Tool (Task 3):")
print(tool_summary)


In [ ]:
# Example DataFrames with different columns
# Assuming df_task1, df_task2, df_task3, df_task4 are already loaded and have different columns

# Step 1: Get the union of all columns across all DataFrames
all_columns = set(df_task1.columns).union(df_task2.columns).union(df_task3.columns).union(df_task4.columns)

# Step 2: Ensure all DataFrames have the same columns by adding missing ones
for df in [df_task1, df_task2, df_task3, df_task4]:
    # Add missing columns with NaN values
    for col in all_columns:
        if col not in df.columns:
            df[col] = pd.NA

# Step 3: Concatenate all DataFrames into one, now they have the same columns
df_all_tasks = pd.concat([df_task2, df_task3, df_task4], ignore_index=True)

# Step 4: Optionally, ensure all numeric columns are converted to numeric if needed
numeric_columns = ['correctness_score', 'complexity_score', 'overall_score']  # Adjust this list based on your columns
for col in numeric_columns:
    if col in df_all_tasks.columns:
        df_all_tasks[col] = pd.to_numeric(df_all_tasks[col], errors="coerce")

# Step 5: Save the final DataFrame to a CSV file
df_all_tasks.to_csv("2-4_combined_task_results.csv", index=False)

# 📊 Show the final DataFrame
print(df_all_tasks)


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load combined results from tasks 2-4
df = pd.read_csv("2-4_combined_task_results.csv")

# Filter necessary columns and drop missing values
df = df[['participant_id', 'tool', 'correctness_score']].dropna()

# Ensure correct data types
df['participant_id'] = df['participant_id'].astype(str)
df['tool'] = df['tool'].astype(str)
df['correctness_score'] = pd.to_numeric(df['correctness_score'], errors='coerce')

# Fit ANOVA model
model = ols('correctness_score ~ C(tool) + C(participant_id)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)
